# Your first convolutional neural network
# imports


In [ ]:
import torch
from torch import nn
from torch.nn.functional import one_hot
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from tqdm.notebook import tqdm

# data

In [ ]:
NUM_CLASSES = 10

In [ ]:
train_data = CIFAR10('~/pytorch', train=True, transform=ToTensor(), download=True)
train_loader = DataLoader(train_data, batch_size=32)
test_data = CIFAR10('~/pytorch', train=False, transform=ToTensor(), download=True)
test_loader = DataLoader(test_data, batch_size=32)

# architecture

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 10, 4, stride=2, padding=1),
    nn.Conv2d(10, 20, 3, stride=2, padding=1),
    nn.Flatten(),
    nn.Linear(8 * 8 * 20, NUM_CLASSES))
print(model)

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 32, 3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    nn.LeakyReLU(),

    nn.Conv2d(32, 32, 3, stride=2, padding=1),
    nn.BatchNorm2d(32),
    nn.LeakyReLU(),
    
    nn.Conv2d(32, 64, 3, stride=1, padding=1),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(),

    nn.Conv2d(64, 64, 3, stride=2, padding=1),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(),

    nn.Flatten(),

    nn.Linear(8 * 8 * 64, 128),
    nn.BatchNorm1d(128),
    nn.LeakyReLU(),
    nn.Dropout(0.5),

    nn.Linear(128, NUM_CLASSES))

# train

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = torch.nn.CrossEntropyLoss(reduction='mean')

In [ ]:
for epoch in range(1, 11):
    total_loss = 0
    pbar = tqdm(train_loader, desc="Epoch {}/10".format(epoch))
    for batch, (x, y) in enumerate(pbar, 1):
        x = x.to(device)
        y = y.to(device)
        output = model(x)
        loss = criterion(output, y)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"loss": "{:.4f}".format(total_loss / ba)})

# analysis

In [ ]:
model.eval()
correct = 0
x = 0.0
y = 1.0
for test_x, test_y in tqdm(test_loader):
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output = model(test_x)
    y_pred = output.argmax(dim=1)
    correct += (y_pred == test_y).float().sum()
print("Accuracy: {:.4f}".format(correct / len(test_data)))

In [ ]:
model.train()